## Task 2 - Is Rectangle?

### Task - Given four positive integers, determine if these four intergers taken as length of sides would form a rectangle.

Solution - Sides of rectangle have a property that parallel sides are equal in length. So for four arbitary intergers to form a rectangle two integers have to be equal to the remaining integers or they could all be euqal forming a square but that is also a case of a rectangle. 

Let the four positive integers be [a,b,c,d]. And 
$$f(x,y) =
  \begin{cases}
    1       & \quad \text{if } x=y \\
    0  & \quad \text{if } x\neq y 
  \end{cases}
$$  
be a equality check between x and y returning 0 is not equal and 1 if equal. We have 4 cases where the four numbers could form a rectangle. The cases are -

1. $a=b \text{ and } c=d$

2. $a=c \text{ and } b=d$

3. $a=d \text{ and } b=d$

4. $a=b=c=d$

We need to check for these cases, we have 6 equalities we have to check for, 6 comes from ${4 \choose 2}$. We can write the above cases in the form of equality checks -

1. $f(a,b)*f(c,d)$

2. $f(a,c)*f(b,d)$

3. $f(a,d)*f(b,c)$

We need to check for the 4th explicitly as we will have all the three check to be 1 in the case of a square. 



#### Checking for equality using SWAP test


In [1]:
import pennylane as qml
import numpy as np
from itertools import combinations

In [2]:
def find_n(feature:list):
    
    max_side = np.max(feature)
    binary = bin(max_side)[2:]
    n = len(binary)

    return n

In [3]:
def is_rectangle(sides:list):
    n_qubits = find_n(sides)

    measurement_qubits = range(3)
    n_measure = len(measurement_qubits)

    n_qubits_circ = n_measure+4*n_qubits

    dev_basis = qml.device('default.qubit',wires=n_qubits_circ+1,shots=100)

    wires_a = range(n_measure,n_qubits+n_measure)
    wires_b = range(n_qubits+n_measure,2*n_qubits+n_measure)
    wires_c = range(2*n_qubits+n_measure,3*n_qubits+n_measure)
    wires_d = range(3*n_qubits+n_measure,4*n_qubits+n_measure)
     
    def swap(wires_0,wires_1,control:int):
        #Comparing a and b - or rather checking if a == b - returns |0> on the ancilla if equal and returns |+>
        for i,j in zip(wires_0,wires_1):
            qml.ctrl(qml.SWAP([i,j]),control=control)

    @qml.qnode(dev_basis)
    def base_enc(feature:list):
    
        qml.BasisEmbedding(features=feature[0], wires=wires_a)
        qml.BasisEmbedding(features=feature[1], wires=wires_b)
        qml.BasisEmbedding(features=feature[2], wires=wires_c)
        qml.BasisEmbedding(features=feature[3], wires=wires_d)
        
    
        qml.Hadamard(measurement_qubits[0])
        swap(wires_a,wires_b,measurement_qubits[0])
        swap(wires_c,wires_d,measurement_qubits[0])
        qml.Hadamard(measurement_qubits[0])

        qml.Hadamard(measurement_qubits[1])
        swap(wires_a,wires_c,measurement_qubits[1])
        swap(wires_b,wires_d,measurement_qubits[1])
        qml.Hadamard(measurement_qubits[1])

        qml.Hadamard(measurement_qubits[2])
        swap(wires_a,wires_d,measurement_qubits[2])
        swap(wires_b,wires_c,measurement_qubits[2])
        qml.Hadamard(measurement_qubits[2])

        return qml.probs(measurement_qubits[0]),qml.probs(measurement_qubits[1]),qml.probs(measurement_qubits[2])
    
    print(qml.drawer.draw(base_enc,max_length=120)(sides))

    circ_result = base_enc(sides)
    rectangle = 0

    for i,j in enumerate(circ_result):
        if  j[0]==1:
            rectangle = rectangle + 1
            
    if rectangle == 3 or rectangle == 1:
        return 1
    return 0



In [6]:

is_rectangle([4,2,2,4])

 0: ──H──────────────╭●────╭●────╭●────╭●────╭●────╭●─────H─────────────────────────────────────────────────────────
 1: ─────────────────│─────│─────│─────│─────│─────│──────H─╭●────╭●────╭●────╭●────╭●────╭●─────H──────────────────
 2: ─────────────────│─────│─────│─────│─────│─────│────────│─────│─────│─────│─────│─────│──────H─╭●────╭●────╭●───
 3: ─╭BasisEmbedding─├SWAP─│─────│─────│─────│─────│────────├SWAP─│─────│─────│─────│─────│────────├SWAP─│─────│────
 4: ─├BasisEmbedding─│─────├SWAP─│─────│─────│─────│────────│─────├SWAP─│─────│─────│─────│────────│─────├SWAP─│────
 5: ─╰BasisEmbedding─│─────│─────├SWAP─│─────│─────│────────│─────│─────├SWAP─│─────│─────│────────│─────│─────├SWAP
 6: ─╭BasisEmbedding─╰SWAP─│─────│─────│─────│─────│────────│─────│─────│─────├SWAP─│─────│────────│─────│─────│────
 7: ─├BasisEmbedding───────╰SWAP─│─────│─────│─────│────────│─────│─────│─────│─────├SWAP─│────────│─────│─────│────
 8: ─╰BasisEmbedding─────────────╰SWAP─│─────│─────│────────│───

1